In [3]:
from datasets import load_dataset

In [4]:
dataset_name = "Ankita802/llm"
dataset = load_dataset(dataset_name)

In [5]:
example_indices = [10, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT QUERY:')
    print(dataset['test'][index]['input'])
    # print(dash_line)
    print('ANSWER:')
    print(dataset['test'][index]['result'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT QUERY:
As a Data Consuming User, I want to see textual descriptions that accompany embedded visualisations, So that I can more easily understand what I am viewing.	

ANSWER:
As a Data Consuming User, I seek textual descriptions accompanying embedded visualizations to enhance my understanding of the content. These descriptions provide valuable context and explanations that aid in comprehending the data presented, ensuring clarity and facilitating informed interpretation of the visual information.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  2
----------------------------------------------------------------------

In [6]:
from transformers import AutoTokenizer, BertModel

model_name =  "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [36]:
sentence = "def groupBy(self, *cols): jgd = self._jdf.groupBy(self._jcols(*cols)) from pyspark.sql.group import GroupedData return GroupedData(jgd, self) "

inputs = tokenizer(sentence, return_tensors="pt")
print(inputs)
decoded_sentence = tokenizer.decode(
            inputs['input_ids'][0],
            skip_special_tokens=True)
print(decoded_sentence)

{'input_ids': tensor([[  101, 13366,  2177,  3762,  1006,  2969,  1010,  1008,  8902,  2015,
          1007,  1024,  1046,  2290,  2094,  1027,  2969,  1012,  1035, 26219,
          2546,  1012,  2177,  3762,  1006,  2969,  1012,  1035, 29175, 27896,
          1006,  1008,  8902,  2015,  1007,  1007,  2013,  1052,  7274, 14432,
          1012, 29296,  1012,  2177, 12324, 15131,  2850,  2696,  2709, 15131,
          2850,  2696,  1006,  1046,  2290,  2094,  1010,  2969,  1007,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
def groupby ( self, * cols ) : jgd = self. _ jdf. gro

without prompt engineering

In [35]:
for i, index in enumerate(example_indices):
    input_query  = dataset['train'][index]['input']
    result = dataset['train'][index]['result']

    inputs = tokenizer(input_query, return_tensors='pt')

    decoded_input = tokenizer.decode(
            inputs['input_ids'][0],
            skip_special_tokens=True)

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{input_query}')
    # print(dash_line)
    print(f'ANSWER FROM CSV:\n{result}')
    print()
    # print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{decoded_input}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
As a user, I want to have access to the content files for an object. 

ANSWER FROM CSV:
As a user, I require access to the content files associated with an object. This feature will enable me to retrieve and utilize the actual content of the object, facilitating tasks such as viewing, editing, or sharing files within the system.

MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
as a user, i want to have access to the content files for an object.

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
As a CONNECT tester, I would like to have a thorough set of regression tests for Document Retrieve, s

without prompt_engineering it will learn only few content from dataset

## ZERO SHOT inference

In [37]:
example_indices = [10, 200]

for i, index in enumerate(example_indices):
    input_query = dataset['train'][index]['input']
    result = dataset['train'][index]['result']

    prompt_template = f"""
   
Providing the description {input_query}
   
   """                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 


    inputs = tokenizer(prompt_template, return_tensors='pt')

    decoded_input = tokenizer.decode(
            inputs['input_ids'][0],
            skip_special_tokens=True)

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{input_query}')
    # print(dash_line)
    print(f'ANSWER FROM CSV:\n{result}')
    print()
    # print(dash_line)
    print(f'MODEL GENERATION - WITH ONE SHOT LEARNING:\n{decoded_input}\n')




---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
As a user, I want to have access to the content files for an object. 

ANSWER FROM CSV:
As a user, I require access to the content files associated with an object. This feature will enable me to retrieve and utilize the actual content of the object, facilitating tasks such as viewing, editing, or sharing files within the system.

MODEL GENERATION - WITH ONE SHOT LEARNING:
providing the description as a user, i want to have access to the content files for an object.

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
As a CONNECT tester, I would like to have a thorough set of regression tests fo

ONE-SHOT Inference

In [18]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        input_query = dataset['train'][index]['input']
        result = dataset['train'][index]['result']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f""" Providing the input query {input_query} Describing the input query {result} """

        dialogue = dataset['train'][example_index_to_summarize]['result']


        prompt += f""" Providing the input query {input_query} Describing the input query {result} """

#     prompt += f"""
# Dialogue:

# {dialogue}

# What was going on?
# """

    return prompt

In [19]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)

 Providing the input query As an Archivist, I want to assign Location information to a Container.
 Describing the input query To assign Location information to a Container as an Archivist, you can follow these steps:

1. Determine the physical location where the Container will be stored. This could be a specific shelf, room, or storage area within your archival facility.

2. Create a unique identifier for the location, such as a shelf number or storage area code, to easily reference the Container's placement.

3. Use a standardized system for recording the location information, such as a database or inventory management software, to ensure consistency and accuracy.

4. Label the Container with its assigned location information, using clear and durable labeling materials to prevent confusion or misplacement.

5. Update any relevant finding aids or catalog records to reflect the assigned location, making it easy for other archivists or researchers to locate the Container when needed.

By

In [26]:

result = dataset['train'][example_index_to_summarize]['result']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')

decoded_input = tokenizer.decode(
            inputs['input_ids'][0],
            skip_special_tokens=True)

print(dash_line)
print('Example ', i + 1)
print(dash_line)
print(f'INPUT PROMPT:\n{input_query}')
    # print(dash_line)
print(f'ANSWER FROM CSV:\n{result}')
print()
    # print(dash_line)
print(f'MODEL GENERATION - WITH ONE SHOT LEARNING:\n{decoded_input}\n')


---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
As a CONNECT tester, I would like to have a thorough set of regression tests for Document Retrieve, so that I can have proper test coverage	

ANSWER FROM CSV:
To establish a comprehensive set of regression tests for Document Retrieve as a CONNECT tester:

Analyze the Document Retrieve feature to understand its functionalities, inputs, and outputs.
Collaborate with stakeholders to identify use cases, requirements, and expected behaviors.
Develop a test plan that covers various scenarios, including:
Retrieving documents by different criteria (e.g., patient ID, document ID, document type)
Testing document retrieval in different formats (e.g., PDF, XML)
Validating document retrieval responses against expected content and metadata
Testing error handling and boundary cas

Few shot inference

In [28]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)

 Providing the input query As an Archivist, I want to assign Location information to a Container.
 Describing the input query To assign Location information to a Container as an Archivist, you can follow these steps:

1. Determine the physical location where the Container will be stored. This could be a specific shelf, room, or storage area within your archival facility.

2. Create a unique identifier for the location, such as a shelf number or storage area code, to easily reference the Container's placement.

3. Use a standardized system for recording the location information, such as a database or inventory management software, to ensure consistency and accuracy.

4. Label the Container with its assigned location information, using clear and durable labeling materials to prevent confusion or misplacement.

5. Update any relevant finding aids or catalog records to reflect the assigned location, making it easy for other archivists or researchers to locate the Container when needed.

By

In [33]:
result = dataset['train'][example_index_to_summarize]['result']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')

decoded_input = tokenizer.decode(
            inputs['input_ids'][0],
            skip_special_tokens=True)

print(dash_line)
print('Example ', i + 1)
print(dash_line)
print(f'INPUT PROMPT:\n{input_query}')
    # print(dash_line)
print(f'ANSWER FROM CSV:\n{result}')
print()
    # print(dash_line)
print(f'MODEL GENERATION - WITH FEW SHOT LEARNING:\n{decoded_input}\n')

---------------------------------------------------------------------------------------------------
Example  2
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
As a CONNECT tester, I would like to have a thorough set of regression tests for Document Retrieve, so that I can have proper test coverage	

ANSWER FROM CSV:
To establish a comprehensive set of regression tests for Document Retrieve as a CONNECT tester:

Analyze the Document Retrieve feature to understand its functionalities, inputs, and outputs.
Collaborate with stakeholders to identify use cases, requirements, and expected behaviors.
Develop a test plan that covers various scenarios, including:
Retrieving documents by different criteria (e.g., patient ID, document ID, document type)
Testing document retrieval in different formats (e.g., PDF, XML)
Validating document retrieval responses against expected content and metadata
Testing error handling and boundary cas